# Redes Neuronales Recurrentes

By Elías Jesús Ventura-Molina

### Importar TensorFlow 2.0  y otras librerias

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

### Obtiene el conjunto de datos

In [2]:
# Ruta donde se encuentra el archivo de texto
path_to_fileDL = 'TextoBase.txt'

# Se lee el archivo
text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

#Se obtiene todos los caracteres que forman el archivo
# Y despues se ordenan
vocab = sorted(set(text))
print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        203249 carácteres
El texto está compuesto de estos 92 carácteres:
['\n', '\r', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


## Pre-procesamiento datos

Las redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica

In [3]:
# Funcion que asigna un valor numerico a cada caracter
char2idx = {u:i for i, u in enumerate(vocab)}
# Funcion que devuelve caracter con base en un numero  
idx2char = np.array(vocab)

#Se  imprime las asocianes caracter-numero
for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '%' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  '4' :  20,
  '5' :  21,
  '6' :  22,
  '7' :  23,
  '8' :  24,
  '9' :  25,
  ':' :  26,
  ';' :  27,
  '<' :  28,
  '=' :  29,
  '>' :  30,
  '?' :  31,
  '@' :  32,
  'A' :  33,
  'B' :  34,
  'C' :  35,
  'D' :  36,
  'E' :  37,
  'F' :  38,
  'G' :  39,
  'H' :  40,
  'I' :  41,
  'J' :  42,
  'K' :  43,
  'L' :  44,
  'M' :  45,
  'N' :  46,
  'O' :  47,
  'P' :  48,
  'Q' :  49,
  'R' :  50,
  'S' :  51,
  'T' :  52,
  'U' :  53,
  'V' :  54,
  'W' :  55,
  'X' :  56,
  'Y' :  57,
  '[' :  58,
  ']' :  59,
  '_' :  60,
  'a' :  61,
  'b' :  62,
  'c' :  63,
  'd' :  64,
  'e' :  65,
  'f' :  66,
  'g' :  67,
  'h' :  68,
  'i' :  69,
  'j' :  70,
  'k' :  71,
  'l' :  72,
  'm' :  73,
  'n' :  74,
  'o' :  75,
  'p' :  76,

Convierte todo el conjutno de datos a su representación numerica

In [4]:
text_as_int = np.array([char2idx[c] for c in text])

Ejemplo de transformación de texto a su representación numerica

In [5]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fun'
array([48, 78, 75, 72, 75, 67, 75,  1,  0, 37, 74,  2, 17, 25, 21, 19, 12,
        2, 41, 79, 61, 61, 63,  2, 33, 79, 69, 73, 75, 82,  2, 76, 81, 62,
       72, 69, 63, 75,  2, 51, 65, 67, 81, 74, 64, 61,  2, 38, 81, 74])


### Preparar los datos para entrenar la RNN


Para entrenar el modelo prepararemos unas secuencias de caracteres como entradas y salida de un tamaño determinado. 


Empezamos dividiendo el texto en secuencias

In [6]:
#Se crea un iterados sobre los datos
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
#Tamaño de la secuencia (puede ser modificado) 
seq_length = 100
#Genera las secuencias
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



Ejemplo de secuencias

In [7]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
'(o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por'
' primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su pri'
'mera escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccio'
'n que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando '
'un ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo,'
' que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de'
' nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de '
'aplicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Proces'
'sor Units), que solo cuestan alrededor de 100 euros, que esta

De esta secuencia se obtiene el conjunto de datos de training que contenga tanto los datos de entrada (desde la posición 0 a la 99) como los datos de salida (desde la posición 1 a la 100). 

La idea es crear patrones donde las entradas y salidas contengan la misma longitud de texto, excepto que la salida es el desplazamiento de la entranada en un carácter a la derecha. 
Ejemplo: la secuencia de entrada será “Hol”, y la de salida será “ola”.

In [8]:
# Se crea la función que genera el conjunto de entrada así como el conjunto de 
# salida con base en una secuencia
def split_input_target(chunk):
  #Define el conjunto de entrada
    input_text = chunk[:-1]
  #Define el conjunto de salida
    target_text = chunk[1:]
  #Retorna ambos conjuntos
    return input_text, target_text

dataset = sequences.map(split_input_target)


Ejemplo de una secuencia generada

In [9]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion'
Target data: 'rologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '


Información del sencuencias generadas

In [10]:
print (dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


Se define parametros de la red

In [11]:
#Tamaño del batch
BATCH_SIZE = 64
#Tamaño del buffer
BUFFER_SIZE = 10000
# Se agrupo los datos en batch y
#Se realiza un acomodo aleatorio
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


## Construcción del modelo

La red esta compuesta por 3 capas:

- La primera capa es de tipo Word Embedding; mapea cada carácter de entrada en un vector Embedding.
Parametros: 
  - Tamaño del vocabulario, indicando cuantos vectores Embedding tendrá la capa
  -Dimension del  vector Embedding 
  -Tamaño del batch 

- La segunda capa es de tipo RRN. Parametros: 
  - Número de neuronas recurrentes
  - Return_sequencese indica que queremos predecir el carácter siguiente a todos los caracteres de entrada, no solo el siguiente al último carácter.
  - Stateful indica el uso de las capacidades de memoria de la red entre batche; si está a true está indicando para cada batch se mantendrán las actualizaciones hechas durante la ejecución del bach anterior.
  - recurrent_kernel indica cómo se deben inicializar los pesos de las matrices internas de la red. En este caso usamos la distribución uniforme glorot_uniform, habitual en estos casos.

- La última capa es de tipo Dense. Parametros:
    - Units indica cuantas neuronas tendrá la capa y que nos marcará la dimensión de la salida. En nuestro caso será igual al tamaño de nuestro vocabulario (vocab_size).



In [12]:
# Función que crea el modelo de la red
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [26]:
#Se difine parametros del modelo
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [14]:
#Se crea el modelo
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [15]:
#Se visualiza la estructura del modelo:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23552     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 92)            94300     
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


## Entrenamiento del modelo

La función de perdida, también conocida como función de costo, es la función que nos dice que tan buena es la red neuronal, un resultado alto indica que la red neuronal tiene un desempeño pobre y un resultado bajo indica que la red neuronal esta haciendo un buen trabajo. Esta es la función que optimizamos o minimizamos cuando realizamos el backpropagation.

In [16]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

Se compila el modelo

In [17]:
model.compile(optimizer='adam', loss=loss)

Configuración de los *checkpoints*

In [18]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

*Training*

In [19]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
31/31 [==============================] - 2s 66ms/step - loss: 3.2853
Epoch 2/50
31/31 [==============================] - 2s 65ms/step - loss: 2.8615
Epoch 3/50
31/31 [==============================] - 2s 65ms/step - loss: 2.5394
Epoch 4/50
31/31 [==============================] - 2s 65ms/step - loss: 2.3220
Epoch 5/50
31/31 [==============================] - 2s 66ms/step - loss: 2.1973
Epoch 6/50
31/31 [==============================] - 2s 66ms/step - loss: 2.1055
Epoch 7/50
31/31 [==============================] - 2s 66ms/step - loss: 2.0138
Epoch 8/50
31/31 [==============================] - 2s 66ms/step - loss: 1.9267
Epoch 9/50
31/31 [==============================] - 2s 66ms/step - loss: 1.8298
Epoch 10/50
31/31 [==============================] - 2s 66ms/step - loss: 1.7427
Epoch 11/50
31/31 [==============================] - 2s 66ms/step - loss: 1.6571
Epoch 12/50
31/31 [==============================] - 2s 67ms/step - loss: 1.5767
Epoch 13/50
31/31 [==================

## Prueba del modelo -  Generación de texto

Se crea el modelo con los últimos pesos generado s

In [20]:
#Se obtiene los ultimos pesos generado
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_50'

In [21]:
# Se define los parametros del modelo
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
# Se carga los ultimos pesos
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
# Se construye los modelos
model.build(tf.TensorShape([1, None]))


Ahora que tenemos el modelo entrenado y preparado para usar, generaremos texto a partir de una palabra de partida con el siguiente código:



In [28]:
def generate_text(model, start_string):

#definir el número de caracteres a predecir 
  num_generate = 500
#convertir la palabra inicial a su correspondiente representación numérica 
  input_eval = [char2idx[s] for s in start_string]
#preparan lo tensores necesarios
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


#Usando la misma idea del código original char-rnn de Andrey Karpathy,
#se usa una variable temperature para decidir que tan permisibo se comporte 
#el modelo.En  este ejemplo la hemos inicializado a 0.5:
  temperature = 0.5
#Con “temperaturas altas” (hasta 1) se permitirá más creatividad al modelo
#para generar texto pero a costa de más errores (por ejemplo, errores ortográficos, etc.). 
# Mientras que con “temperaturas bajas” habrá menos errores pero el modelo mostrará poca creatividad.


  model.reset_states()
  for i in range(num_generate):
    #Realiza la prediccion 
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

    #Se aplica la temperatura a la predicción 
      predictions = predictions / temperature
   #Se usa una distribución categórica para calcular el índice del carácter predicho:
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    #Este carácter acabado de predecir se usa como nuestra próxima entrada al modelo
      input_eval = tf.expand_dims([predicted_id], 0)

  #La prediccion es almacenada es una variable
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))





### Ejemplos de generación de texto

Los siguientes son ejemplos de corrida de la función anterior

In [30]:
print(generate_text(model, start_string=u"modelo"))

modelo deben ser derivablos por los datos de entrenamiento para almacenar los datos disponibles en total de cada uno de ellos separadamente, la capa de pooling con una se de la pantalla de una funcion que adante la probabilidad esto es una experto en Machine Learning, pero esta visto que estos ellos despues de entrada.
*  frece de GPUs esta informacion para tiene un supercomputacion presentan un ejemplo de red neuronal convolucional que el proceso de aprendizaje por un escalar conocido como el parame


In [31]:
print(generate_text(model, start_string=u"activacion"))

activacion para entrenar a los modelos de inteligencia artificial en general, en concreto, del modelo en particular del tema del libro una vez incluidos en la entrada de 28?28 pixeles y una ventana de 5?5 en la capa de pooling contendra una red neuronal artificial con el metodo summary() que retorna el metodo summary() podemos encontrar todos los tipos de problema. Adjunto al codigo del libro hablaremos mas de un tensor 3D de puesto esta es que estamas habitualmente las redes neuronales convolucionales po


In [33]:
print(generate_text(model, start_string=u"perro"))

perro descargar de GitHub localmente a probar los hallanzos esta compuesto por multiples capas ocultas (hidden layers) y podemos tener hasta ocho neuronas tecnicas de redes neuronales convolucionales para que los valores iniciales de la primera capa de pooling, que sueve elemento el lector, vientros de entrada, se han refire con el tema,  desde hace un tiempo en el resultado apartado varias operaciones por se este es demasiado para tar entender los conceptos basicos de este si nos en este caso, el mod
